# Naive Bayes

Naive Bayes is supervised learning algorithm that uses the probabilities of class attributes to make a prediction.

The name *Naive* comes from a strong assumption that the probability of each attribute belonging to a given class is independent of all other attributes.

The probability of a class given a value of an attribute is called the *conditional probability*. The product of the *conditional probabilities* of each attribute for a give class yields a probability of a data instance belonging to that class.

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data

The data used is the [Pima Indians Diabetes](https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes) problem:

* 768 Pima Indian female patients aged 21 or older.
* Each data instance contains numeric-only information such as age, number of times pregnant, blood pressure etc.
* A binary classification problem:
    * 1 if the patient suffered an onset of diabetes within 5 years of when the measurements were taken.
    * 0 otherwise.
    
The description of each attribute can found [here](https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.names).

In [5]:
data = pd.read_csv(
    'pima_indians_diabetes.csv',
    header=None,
    names=[
        'Number of times pregnant',
        'Plasma glucose concentration',
        'Diastolic blood pressure',
        'Triceps skin fold thickness',
        '2-Hour serum insulin',
        'Body mass index',
        'Diabetes pedigree function',
        'Age',
        'Class' # 1 - diabetes, 0 - otherwise
    ]
)

data.describe().transpose()[['count', 'mean', 'std']]

,count,mean,std
Number of times pregnant,768.0,3.845052,3.369578
Plasma glucose concentration,768.0,120.894531,31.972618
Diastolic blood pressure,768.0,69.105469,19.355807
Triceps skin fold thickness,768.0,20.536458,15.952218
2-Hour serum insulin,768.0,79.799479,115.244002
Body mass index,768.0,31.992578,7.884160
Diabetes pedigree function,768.0,0.471876,0.331329
Age,768.0,33.240885,11.760232
Class,768.0,0.348958,0.476951


In [13]:
print('1 labels: %d' % data['Class'].sum())
print('0 labels: %d' % (len(data) - data['Class'].sum()))

1 labels: 268
0 labels: 500
